In [1]:
# Setting up the environment

import svelte_widget
import xgboost
import shap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

2023-05-22 12:58:25.128578: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-22 12:58:25.279422: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-22 12:58:25.280982: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 12:58:26.688791: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Upload and set up data

from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

df = pd.read_csv("BRCA.csv")

# Deal with non-numerical variables

df = df.drop(df.columns[0], axis=1)
df['Gender'] = df['Gender'].map({"FEMALE": 0, "MALE": 1})
df['Tumour_Stage'] = df['Tumour_Stage'].map({"I": 0, "II": 1, "III": 2})
df['Histology'] = df['Histology'].map({"Infiltrating Ductal Carcinoma": 0, "Mucinous Carcinoma": 1})
df['ER status'] = df['ER status'].map({"Negative": 0, "Positive": 1})
df['PR status'] = df['PR status'].map({'Negative': 0, 'Positive': 1})
df['HER2 status'] = df['HER2 status'].map({"Negative": 0, "Positive": 1})
df['Surgery_type'] = df['Surgery_type'].map({"Modified Radical Mastectomy": 0, "Lumpectomy": 1, "Simple Mastectomy": 2})
df['Patient_Status'] = df['Patient_Status'].map({"Dead": 0, "Alive": 1})
# Temporal variables for now are not considered
'''
for i in range(0, df.shape[0]):
    df.loc[i, 'Date_of_Surgery'] = datetime.strptime(df.loc[i, 'Date_of_Surgery'], '%d-%b-%y')
    df.loc[i, 'Date_of_Last_Visit'] = datetime.strptime(df.loc[i, 'Date_of_Last_Visit'], '%d-%b-%y')
'''
df = df.drop(['Date_of_Surgery', 'Date_of_Last_Visit'], axis=1)

df = df.dropna()

# Initialize the scaler
scaler = MinMaxScaler()

# Apply the scaler to every column in the dataframe
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

X = df.drop(columns=['Patient_Status'])

# We do not want to scale Y, so widget will display real values
# for predictions and not scaled ones
y = df['Patient_Status']

In [3]:
# Defining and training two Neural Networks

from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

import tensorflow as tf
from tensorflow import keras

# Define the first neural network model
model_1 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_1.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Define the second neural network model
model_2 = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_2.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

2023-05-22 12:58:28.539724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-22 12:58:28.540334: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10
5/5 [==============================] - 1s 75ms/step - loss: 0.6353 - accuracy: 0.8529 - val_loss: 0.6561 - val_accuracy: 0.6571
Epoch 2/10
5/5 [==============================] - 0s 17ms/step - loss: 0.5887 - accuracy: 0.8529 - val_loss: 0.6476 - val_accuracy: 0.6571
Epoch 3/10
5/5 [==============================] - 0s 18ms/step - loss: 0.5491 - accuracy: 0.8529 - val_loss: 0.6442 - val_accuracy: 0.6571
Epoch 4/10
5/5 [==============================] - 0s 21ms/step - loss: 0.5127 - accuracy: 0.8529 - val_loss: 0.6490 - val_accuracy: 0.6571
Epoch 5/10
5/5 [==============================] - 0s 16ms/step - loss: 0.4802 - accuracy: 0.8529 - val_loss: 0.6644 - val_accuracy: 0.6571
Epoch 6/10
5/5 [==============================] - 0s 18ms/step - loss: 0.4534 - accuracy: 0.8529 - val_loss: 0.6920 - val_accuracy: 0.6571
Epoch 7/10
5/5 [==============================] - 0s 16ms/step - loss: 0.4315 - accuracy: 0.8529 - val_loss: 0.7321 - val_accuracy: 0.6571
Epoch 8/10
5/5 [===========

In [4]:
z = svelte_widget.ExampleWidget(X_test,y_test,model_1.predict,model_2.predict)
z

Start...
Predictions obtained
2/2 [==============================] - 0s 4ms/step
Shaps now
Shaps XGB
Shaps NN
Calculating...
-> m1
10/10 [==============================] - 0s 2ms/step


Permutation explainer:   9%|█████▉                                                               | 3/35 [00:00<?, ?it/s]

11/11 [==============================] - 0s 1ms/step


Permutation explainer:  14%|████████▋                                                    | 5/35 [00:14<00:42,  1.42s/it]

10/10 [==============================] - 0s 2ms/step


...

Permutation explainer: 100%|████████████████████████████████████████████████████████████| 35/35 [01:40<00:00,  2.94s/it]

10/10 [==============================] - 0s 2ms/step


Permutation explainer: 36it [01:43,  3.24s/it]                                                                          

Difference...
Done


ExampleWidget(dataset={'Age': [0.13793103448275867, 0.6724137931034482, 0.5862068965517242, 0.5344827586206897…